<h1> Polynomial regression in 1D

So you have some data, and you'd like to get a "best-fit" curve through the data. No problem. That's pretty much what this guide seeks to do. So let's get started; consider the data set below.

$$\begin{array}{cc} \hline
  & X &  Y  \\ \hline
         0  &  6.8053 \\
    0.0714  & -1.5184 \\
    0.1429  &  1.6416 \\
    0.2143  &  7.0774 \\
    0.2857  &  6.3543 \\
    0.3571  &  14.3442 \\
    0.4286  & 16.4426 \\
    0.5000  &  17.6663 \\
    0.5714  &  18.1953 \\
    0.6429  &  28.9913 \\
    0.7143  &  27.2246 \\
    0.7857  &  40.3759 \\
    0.8571  &  49.8868 \\
    0.9286  &  55.3726 \\
    1.0000  &  66.5187 \\ \hline
\end{array}$$

Our goal is to compute the best fit quadratic polynomial approximation through this data set. We begin by calling two classes from the code. The first class is the Parameter class, that handles all the information regarding the independent variable, or parameter. The second class is the Polynomial class, which actually generates the "best-fit" curve using the information in the parameter. 

In [1]:
%matplotlib inline
from effective_quadratures.parameter import Parameter
from effective_quadratures.polynomial import Polynomial
import numpy as np

The first step is to set the limits on the x's. We can assume that x is defined on [0,1].

In [5]:
x = Parameter(lower=0, upper=1)

Now, compile our data into a matrix form:

In [3]:
x_train = [0,0.0714,0.1429,0.2143,0.2857,0.3571,0.4286,0.5000,0.5714,0.6429,0.7143,0.7857,0.8571,0.9286,1.0000]
y_train = [6.8053,-1.5184,1.6416,7.0774,6.3543,14.3442,16.4426,17.6663,18.1953,28.9913,27.2246,40.3759,49.8868,55.3726,66.5187]

Now, we call the "poly

In [4]:
esq = EffectiveSubsampling(parameters, polybasis)
points = esq.getEffectivelySubsampledPoints(maximum_number_of_evals)
x = esq.solveLeastSquares(maximum_number_of_evals, piston)

NameError: name 'EffectiveSubsampling' is not defined

Printing the output to the screen:

In [ ]:
pistonUQ = Statistics(x, polybasis)
mean = pistonUQ.getMean()
variance = pistonUQ.getVariance()
sobol = pistonUQ.getFirstOrderSobol()

print 'VARIANCE'
print variance
print '\n'
print 'FIRST ORDER SOBOL INDICES:'
print sobol
print '\n'
print 'NORMALIZED INDICES'
print sobol * variance * 10**2

These values closely match those in Owen et al. [3]. 

<h3> References

1. Blatman, Géraud, and Bruno Sudret. "Adaptive sparse polynomial chaos expansion based on least angle regression." Journal of Computational Physics 230.6 (2011): 2345-2367.
2. Kenett, Ron, Shelemyahu Zacks, and Daniele Amberti. Modern Industrial Statistics: with applications in R, MINITAB and JMP. John Wiley & Sons, 2013.
3. Owen, Art B., Josef Dick, and Su Chen. "Higher order Sobol'indices." Information and Inference 3.1 (2014): 59-81.